## BERT MODEL

In [5]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.linear_model import Ridge, Lasso
from sklearn.utils import resample
import pandas as pd
import xgboost
import numpy as np 
import nltk
import gensim 
from gensim.models import Word2Vec
from keras.preprocessing.text import text_to_word_sequence
import string
from nltk import word_tokenize
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


Read Dataset

In [6]:
nn = pd.read_excel('C:\\Users\\bshukla\\baseline.xlsx')
nn['tokens'] = nn['q1_q2'].apply(word_tokenize) 
nn = nn.astype(str)
nn = nn.sample(n = 5000, random_state=1) 
nn
nn.is_duplicate.value_counts()

0    3108
1    1892
Name: is_duplicate, dtype: int64

BALANCE DATASET

In [7]:
nnn_majority = nn[nn.is_duplicate=='0']
nnn_minority = nn[nn.is_duplicate=='1']


nnn_minority
df_majority_downsampled = resample(nnn_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=1892,     # to match minority class
                                 random_state=123) # reproducible results
nn = pd.concat([df_majority_downsampled, nnn_minority])

nn.is_duplicate.value_counts()

0    1892
1    1892
Name: is_duplicate, dtype: int64

In [8]:
nn['is_duplicate'].value_counts()

0    1892
1    1892
Name: is_duplicate, dtype: int64

In [12]:
from simpletransformers.classification import MultiLabelClassificationModel

df = nn[["q1_q2","is_duplicate"]]
df['is_duplicate'].value_counts()

0    1892
1    1892
Name: is_duplicate, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
df

,q1_q2,is_duplicate
331007,mr robot tv series is mr robot a good represen...,0
190780,what were the most famous sportsgames in india...,0
244480,how do i stop a redirecting virus on android i...,0
139526,how can i make curly hair grow faster and long...,0
264734,what does this joke meanwhat is the meaning of...,0
...,...,...
394683,what does it take to be a car salesmanhow do y...,1
390605,why do tall guys love short girlsis it true th...,1
249757,why do people believe in conspiracy theorieswh...,1
144779,which universities in canada offer diploma in ...,1


SPLIT INTO TRAIN & VALIDATION

In [23]:
df

train, test = train_test_split(df, test_size=0.5, stratify = df['is_duplicate'])


train['q1_q2'].apply(str)
test['q1_q2'].apply(str)

train["is_duplicate"] = pd.to_numeric(train["is_duplicate"])
test["is_duplicate"] = pd.to_numeric(test["is_duplicate"])

test.dtypes

C:\Users\bshukla\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\bshukla\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


q1_q2           object
is_duplicate     int64
dtype: object

BERT MODEL IMPLEMENTATION

In [15]:
from simpletransformers.classification import ClassificationModel

In [18]:
modeltwo = ClassificationModel('bert', 'bert-base-uncased', use_cuda = False, args = {'overwrite_output_dir': True, 'num_train_epochs': 3})
modeltwo.train_model(train)
resulttwo, modeltwo_outputs, wrong_predictions = modeltwo.eval_model(test)

Converting to features started.


Running loss: 0.491029


Running loss: 0.096694


Running loss: 0.020732

Training of bert model complete. Saved to outputs/.
Converting to features started.


BERT MODEL PERFORMANCE

In [27]:
resulttwo, modeltwo_outputs, wrong_predictions = modeltwo.eval_model(train)

Features loaded from cache at cache_dir/cached_dev_bert_128_2_1892


In [20]:
resulttwo

{'mcc': 0.5673987968268284,
 'tp': 871,
 'tn': 587,
 'fp': 359,
 'fn': 75,
 'eval_loss': 0.8151762967512703}

In [30]:
precision = 871/(871+359)
recall = 871/(871+75)
f1_score = (2*precision*recall)/(precision+recall)
## Result 2 performance 
print("Resulttwo Performance: ", 
      "Precision:", precision, 
      "Recall:", recall, 
      "F1 Score:", f1_score)

Resulttwo Performance:  Precision: 0.708130081300813 Recall: 0.9207188160676533 F1 Score: 0.8005514705882353


In [ ]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', use_cuda = False, args = {'overwrite_output_dir': True, 'num_train_epochs': 3})
model.train_model(a)
result, model_outputs, wrong_predictions = model.eval_model(b)